In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import itertools
#from tqdm import tqdm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.feature_selection import SequentialFeatureSelector, f_regression

In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [3]:
df = pd.read_csv('logP.csv')
y = df.logP
X = df.iloc[:, 3:]
feats = X.columns
X

,QED_score,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0.909,12.053455,12.053455,0.045556,0.045556,0.909253,281.355,262.203,281.141579,108,...,0,0,0,0,0,0,0,0,0,0
1,0.839,12.616874,12.616874,0.009514,-5.052156,0.839542,262.183,253.111,262.045293,98,...,0,0,0,0,0,0,0,0,0,0
2,0.828,4.217020,4.217020,1.056122,1.056122,0.828333,266.388,244.212,266.178299,104,...,0,0,0,0,0,0,0,0,0,0
3,0.726,11.239605,11.239605,0.364327,-1.508792,0.726080,318.372,300.228,318.125594,120,...,0,0,0,0,0,0,0,0,0,0
4,0.816,4.352931,4.352931,0.140811,0.140811,0.816043,214.312,196.168,214.146999,84,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.863,12.562081,12.562081,0.093167,-0.455663,0.863227,381.501,358.317,381.151098,140,...,1,0,0,0,0,0,0,0,0,0
240,0.737,12.114099,12.114099,0.072506,-0.211498,0.737878,321.376,302.224,321.136493,122,...,0,0,0,0,0,0,0,0,0,0
241,0.824,12.974979,12.974979,0.339313,-1.732014,0.824192,274.266,262.170,274.080536,102,...,0,0,0,0,0,0,0,0,0,0
242,0.750,12.347197,12.347197,0.125336,-0.724707,0.750035,321.312,308.208,321.088080,120,...,0,0,0,0,0,0,0,0,0,0


In [6]:
model = linear_model.LinearRegression()
sfs = SequentialFeatureSelector(model, cv = kf, n_jobs=-1)  #n_features_to_select=50
sfs.fit(X, y)


SequentialFeatureSelector(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                          estimator=LinearRegression(), n_jobs=-1)

In [7]:
mask = sfs.get_support()
X_new = X[X.columns[mask]]
feats2 = X.columns[mask]
X_new
len(feats2)

104

In [18]:
def feature_selection_cv(X, y, features: list, n_features = 2):

    r2_values = []

    for n_feat in tqdm(range(1, n_features+1)):

        combs = itertools.combinations(features, n_feat)

        for feature_subset in combs:

            model = linear_model.LinearRegression()

            X_train = X[[*feature_subset]]

            coeff = cross_val_score(model, X_train , y, cv = kf).mean()
            r2_values.append((n_feat, coeff, ' '.join(feature_subset)))

    r2_df = pd.DataFrame(r2_values, columns=['n', 'r2', 'cols'])

    return r2_df.groupby('n').max()


In [6]:
res = feature_selection_cv(X_new, y, feats2, 3)
res

NameError: name 'feature_selection_cv' is not defined

In [10]:
cross_val_score(linear_model.LinearRegression(), X_new, y, cv = kf).mean()

0.5890638062403051